In [8]:
import json

path = "/content/drive/MyDrive/Colab Notebooks/BERT_sentiment.ipynb"

with open(path, "r", encoding="utf-8") as f:
    nb = json.load(f)

nb.get("metadata", {}).pop("widgets", None)

with open("/content/BERT_sentiment_clean.ipynb", "w", encoding="utf-8") as f:
    json.dump(nb, f)

print("Clean notebook created!")



FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/BERT_sentiment.ipynb'

In [ ]:
pip install -U transformers datasets torch accelerate evaluate scikit-learn


ERROR: Operation cancelled by user


In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
print(tokenizer("I love machine learning"))


{'input_ids': [101, 1045, 2293, 3698, 4083, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1]}


In [ ]:
from datasets import load_dataset

dataset = load_dataset("imdb")
print(dataset)


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=256)

tokenized_ds = dataset.map(tokenize, batched=True)
tokenized_ds = tokenized_ds.remove_columns(["text"])
tokenized_ds.set_format("torch")

print(tokenized_ds)


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50000
    })
})


In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=2
)

print(model)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [ ]:
from transformers import TrainingArguments, Trainer
import evaluate
import numpy as np

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return metric.compute(predictions=preds, references=labels)

from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="./sentiment_model",
    do_train=True,
    do_eval=True,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    logging_dir="./logs",
    save_steps=500,
    eval_steps=500,
)


trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds["train"].shuffle(seed=42).select(range(20000)),
    eval_dataset=tokenized_ds["test"].shuffle(seed=42).select(range(5000)),
    compute_metrics=compute_metrics
)

trainer.train()


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss
500,0.392000
1000,0.322200
1500,0.301900
2000,0.280700
2500,0.293500
3000,0.177800
3500,0.197500
4000,0.149200
4500,0.180200
5000,0.173700


TrainOutput(global_step=5000, training_loss=0.24687305755615235, metrics={'train_runtime': 2448.6253, 'train_samples_per_second': 16.336, 'train_steps_per_second': 2.042, 'total_flos': 5262221107200000.0, 'train_loss': 0.24687305755615235, 'epoch': 2.0})

In [ ]:
from transformers import pipeline

clf = pipeline(
    "sentiment-analysis",
    model="./sentiment_model",
    tokenizer="bert-base-uncased"
)

print(clf("This movie was an absolute masterpiece!"))
print(clf("The story was boring and terrible."))


Device set to use cuda:0


[{'label': 'LABEL_1', 'score': 0.9988822340965271}]
[{'label': 'LABEL_0', 'score': 0.9990441203117371}]


LABEL_1 --> 1
LABEL_0 --> 0